In [ ]:
import mercantile
import geopandas as gpd

In [ ]:
# Test Tile Coordinates:
# Dinslaken Duisburg etc.
z = 10
x = 531
y = 340

In [ ]:
tile = mercantile.Tile(x,y,z)

In [ ]:
# In latitude longitude:
upper_left = mercantile.ul(tile)
upper_left

In [ ]:
lower_right = mercantile.ul(x+1,y-1,z)
lower_right

In [ ]:
# bbox in meters:
bbox = mercantile.xy_bounds(tile)
bbox

In [ ]:
width = bbox.right - bbox.left
height = bbox.top - bbox.bottom
width,height

# Download the Vector Tile

In [ ]:
import mapbox_vector_tile
import requests

In [ ]:
tileset_id = 'mapbox.mapbox-streets-v8'

In [ ]:
token = 'pk.eyJ1Ijoibmljb2pnIiwiYSI6ImNrcHFqamY5azNqaTAycHJpa210ZDF2aG4ifQ.8jPQpYl4eiUw0Cv_b8J7hA'

In [ ]:
url = f'https://api.mapbox.com/v4/{tileset_id}/{z}/{x}/{y}.mvt?access_token={token}'
url

In [ ]:
file_path_mvt = f'data/vector_tiles/z{z}x{x}y{y}_{tileset_id}.mvt'

In [ ]:
#r = requests.get(url)  
#with open(file_path_mvt, 'wb') as f:
#    f.write(r.content)

In [ ]:
with open(file_path_mvt, mode='rb') as file: # b is important -> binary
    file_mvt = file.read()

In [ ]:
file_mvt

In [ ]:
vector_tile = mapbox_vector_tile.decode(file_mvt)
vector_tile

In [ ]:
vector_tile.keys()

In [ ]:
type(vector_tile)

In [ ]:
# read Vector Tile via python-vt2geojson
from vt2geojson.tools import vt_bytes_to_geojson

vector_tile_ = vt_bytes_to_geojson(file_mvt, x, y, z, layer='water')
vector_tile_

In [ ]:
vector_tile_df = gpd.GeoDataFrame.from_features(vector_tile_,crs='EPSG:4326')
vector_tile_df.plot()

## .mvt to GeoDataFrame with Geodetic coordinates

In [ ]:
# using mapbox_vector_tile and the formula vector_tile to latlon
file_path_mvt = f'data/vector_tiles/z{z}x{x}y{y}_{tileset_id}.mvt'
with open(file_path_mvt, mode='rb') as file: # b is important -> binary
    file_mvt = file.read()

In [ ]:
vector_tile = mapbox_vector_tile.decode(file_mvt)

In [ ]:
for key in vector_tile:
    print(f"'{key}': \t{vector_tile[key].keys()} ,\t extent={vector_tile[key]['extent']}")

In [ ]:
vector_tile_dfs = {key:gpd.GeoDataFrame.from_features(vector_tile[key]['features']) for key in vector_tile}
vector_tile_dfs

In [ ]:
water_polygon = vector_tile_dfs['water']['geometry'].values[0]

In [ ]:
import shapely

In [ ]:
import numpy as np
# https://github.com/Amyantis/python-vt2geojson/issues/8
def vt_to_lonlat(tile_x,tile_y,tile_z,pixel_x,pixel_y):
    """
    Converts Mapbox Vector Tile coordinates to longitude and latitude
    from https://github.com/Amyantis/python-vt2geojson/blob/master/vt2geojson/features.py
    
    Parameters
    ----------
    tile_x : int
        x coordinate of the tile
    tile_y : int
        y coordinate of the tile
    tile_z : int
        zoom of the tile
    pixel_x : int
        x coordinate in the vector tile
    pixel_y : int
        y coordinate in the vector tile
    
    Returns
    -------
    (lon,lat) : transformed coordinates
    """
    tile_extent = 4096
    pixel_y = tile_extent - pixel_y # top left has to be 0,0
    size = tile_extent * 2**tile_z
    x0 = tile_extent * tile_x
    y0 = tile_extent * tile_y
    lon = (x0 + pixel_x) * 360. / size - 180.
    y2 = 180. - (y0 + pixel_y) * 360. / size
    lat = 360. / np.pi * np.arctan(np.exp(y2 * np.pi / 180.)) - 90.
    return lon,lat

In [ ]:
water_polygon_lonlat = shapely.ops.transform(lambda px, py: vt_to_lonlat(x,y,z,px,py),water_polygon)
water_polygon_lonlat

In [ ]:
water_polygon

In [ ]:
gpd.GeoDataFrame(crs='EPSG:4326',geometry=[water_polygon_lonlat]).plot()

In [ ]:
vector_tile_dfs['water'].plot()

In [ ]:
for key in vector_tile_dfs:
    vector_tile_dfs[key]['geometry'] = shapely.ops.transform(lambda px, py: vt_to_lonlat(x,y,z,px,py),water_polygon)

# Inspect Vector Tile

In [ ]:
vector_tile.keys()

In [ ]:
print(water_df.crs)

In [ ]:
water_df = gpd.GeoDataFrame.from_features(vector_tile['water']['features'])

In [ ]:
water_df

In [ ]:
water_df.plot(figsize=(10,10))

In [ ]:
vector_tile_df.plot(figsize=(10,10))

# Check if a given Tile inside the larger file has Water in it

In [ ]:
# Rotbachsee Dinslaken s für small
z_s = 15
x_s = 17001
y_s = 10887

In [ ]:
tile_s =  mercantile.Tile(x_s,y_s,z_s)

In [ ]:
# In latitude longitude:
upper_left = mercantile.ul(tile_s)
lower_right = mercantile.ul(x_s+1,y_s-1,z_s)
upper_left, lower_right

In [ ]:
# bbox in meters:
bbox = mercantile.xy_bounds(tile_s)
bbox

In [ ]:
width = bbox.right - bbox.left
height = bbox.top - bbox.bottom
width,height

# Using GDAL / ogr2ogr to decode Mapbox Vector Tiles

In [ ]:
# Test Tile Coordinates:
# Dinslaken Duisburg etc.
z = 10
x = 531
y = 340

In [ ]:
file_path_mvt = f'data/vector_tiles/z{z}x{x}y{y}_{tileset_id}.mvt'

In [ ]:
tileset_id = 'mapbox.mapbox-streets-v8'

In [ ]:
file_path_json = f'data/vector_tiles/z{z}x{x}y{y}_{tileset_id}.json'

In [ ]:
!ogr2ogr -f GeoJSON $file_path_json $file_path_mvt -oo x=$x -oo y=$y -oo z=$z

In [ ]:
file_path_shp = f'data/vector_tiles/z{z}x{x}y{y}_{tileset_id}.shp'

In [ ]:
!ogr2ogr -f "ESRI Shapefile" $file_path_shp $file_path_mvt -oo x=$x -oo y=$y -oo z=$z

# Working in Pixel Coordinates

### import stuff

In [ ]:
import mercantile
import requests
import mapbox_vector_tile
import geopandas as gpd
import shapely.geometry
import matplotlib.pyplot as plt

### define the coordinates and calculate the tile coordinates

$x_i,y_i,z_i$ are the Tile Coordinates of the inner Tile (which will be the Image Tile)  
$x_o,y_o,z_o$ are the Tile Coordinates of the outer Tile

In [ ]:
# Test Coordinates:
# Dinslaken Rotbachsee
lat,lon = 51.56792718343954, 6.78504175412729
# Dortmund Phönixsee
#lat, lon = 51.49011058141199, 7.506262816458572
# Tenderingssee
lat,lon = 51.59932699198615, 6.719474067819083
z_i = 15
z_o = 10

In [ ]:
tile_i = mercantile.tile(lon,lat,z_i)

In [ ]:
x_i,y_i = tile_i.x,tile_i.y
z_i,x_i,y_i

In [ ]:
tile_o = mercantile.tile(lon,lat,z_o)

In [ ]:
x_o,y_o = tile_o.x,tile_o.y
z_o,x_o,y_o

### Download and read in the Vector Tile of the outer Tile

In [ ]:
tileset_id = 'mapbox.mapbox-streets-v8'

In [ ]:
token = 'pk.eyJ1Ijoibmljb2pnIiwiYSI6ImNrcHFqamY5azNqaTAycHJpa210ZDF2aG4ifQ.8jPQpYl4eiUw0Cv_b8J7hA'

In [ ]:
file_path_mvt = f'data/vector_tiles/z{z_o}x{x_o}y{y_o}_{tileset_id}.mvt'
file_path_mvt

In [ ]:
# download the vector tile if it's not already there
import os.path
if not os.path.isfile(file_path_mvt):
    url = f'https://api.mapbox.com/v4/{tileset_id}/{z_o}/{x_o}/{y_o}.mvt?access_token={token}'
    print(url)
    #r = requests.get(url)  
    #with open(file_path_mvt, 'wb') as f:
    #    f.write(r.content)
else:
    print(f'MVT file already exists')

In [ ]:
# read the vector tile
with open(file_path_mvt, mode='rb') as file:
    vector_tile = mapbox_vector_tile.decode(file.read(),y_coord_down=True)

In [ ]:
vector_tile.keys()

In [ ]:
water_df = gpd.GeoDataFrame.from_features(vector_tile['water']['features'])
water_df

### calculate the bounding box of the inner tile in vector tile pixel coordinates

https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames  
https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/  

In [ ]:
# pixel width and height of the outer vector tile
extent_o = 4096

In [ ]:
# pixel width and height of one inner tile
extent_i = extent/(2**(z_i-z_o))

In [ ]:
# top left Tile in the outer Tile but with the inner zoom
# https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Subtiles
x_o_with_z_i = 2**(z_i-z_o) * x_o
y_o_with_z_i = 2**(z_i-z_o) * y_o

In [ ]:
# pixel coordinates of the top left corner of the inner tile in the vector tile
px_tl = extent_i * (x_i - x_o_with_z_i)
py_tl = extent_i * (y_i - y_o_with_z_i)

In [ ]:
# bottom right corner of the inner tile in vector tile pixel coordinates
px_br = px_tl + extent_i
py_br = py_tl + extent_i

In [ ]:
bbox = shapely.geometry.box(px_tl, py_tl, px_br, py_br)

In [ ]:
print(bbox)

In [ ]:
ax = water_df.plot()
gpd.GeoSeries(bbox).plot(ax=ax,color='red')
plt.gca().invert_yaxis()

In [ ]:
water_df.plot()
plt.xlim(px_tl,px_br)
plt.ylim(py_tl,py_br)

### check if there is water in the bounding box

In [ ]:
# Geometries overlaps if they have more than one but not all points in common, 
# have the same dimension, 
# and the intersection of the interiors of the geometries has the same dimension as the geometries themselves.
water_df.overlaps(bbox)